## Итоговое задание  №4

<b>1.</b> Сгруппируйте данные по IP и посчитайте количество запросов для каждого IP, выводим 10 самых активных IP. Формат вывода, как на скрине ниже.

<b>2.</b> Сгруппируйте данные по HTTP-методу и посчитайте количество запросов для каждого метода.

<b>3.</b> Профильтруйте и посчитайте количество запросов с кодом ответа 404.

<b>4.</b> Сгруппируйте данные по дате и просуммируйте размер ответов, сортируйте по дате.

In [20]:
# инициализация Spark сессии
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession,DataFrame
from pyspark.sql.functions import col, year, month, sum, avg, count
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, TimestampType

conf = (
     SparkConf()
    .setAppName('Simple_Spark')
    .setMaster('local[*]')
)


# Spark сессия
spark = (
    SparkSession
    .builder
    .config(conf=conf)
    # .enableHiveSupport()
    .getOrCreate()
)


In [21]:
# схема данных
schema = (StructType([StructField('ip', StringType(), True),
        StructField('timestamp', TimestampType(), True),
        StructField('method', StringType(), True),
        StructField('url', StringType(), True), 
        StructField('response_code', IntegerType(), True),
        StructField('response_size', IntegerType(), True)
       ])
        )

In [22]:
# загрузка данных в df из файла .csv, проверка корректности данных по 5 строкам
# Внимание! для загрузки поменять путь на свой, если папка data перемещалась
file = "./data/web_server_logs.csv"

df = spark.read.csv(file, header=True, schema=schema)
df.show(5, 100, False)

+--------------+--------------------------+------+---------------------+-------------+-------------+
|            ip|                 timestamp|method|                  url|response_code|response_size|
+--------------+--------------------------+------+---------------------+-------------+-------------+
|40.110.212.214|2024-08-11 04:54:47.693419|  POST|           categories|          500|         2067|
| 19.130.93.143|2024-05-24 01:51:11.620188|   PUT|tags/explore/category|          301|         2029|
|142.12.126.182|2024-02-14 08:14:25.862731|   GET|posts/wp-content/tags|          404|         6135|
| 130.46.123.96|2024-08-13 23:03:17.545586|  POST|       wp-content/app|          404|         5517|
|203.215.14.169|2024-01-18 03:09:53.438453|   GET|                  app|          404|         2124|
+--------------+--------------------------+------+---------------------+-------------+-------------+
only showing top 5 rows



In [23]:
# проверим схему данных:
df.printSchema()

root
 |-- ip: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- method: string (nullable = true)
 |-- url: string (nullable = true)
 |-- response_code: integer (nullable = true)
 |-- response_size: integer (nullable = true)



In [24]:
# проверим null в колонках
df.filter(col("ip").isNull()).count()

0

1. `Сгруппируйте данные по IP и посчитайте количество запросов для каждого IP, выводим 10 самых активных IP.`

In [25]:
task1_df = (
    df
    .groupBy("ip")
    .agg(count("*").alias("request_count"))
    .orderBy(col("request_count").desc())
    .limit(10)
)

print("Top 10 active IP addresses: ")
task1_df.show()

Top 10 active IP addresses: 
+---------------+-------------+
|             ip|request_count|
+---------------+-------------+
| 169.224.123.25|            2|
|  33.213.65.240|            2|
|  169.136.2.244|            1|
| 172.254.175.14|            1|
|  167.130.16.96|            1|
|    9.27.24.117|            1|
|215.244.100.237|            1|
| 102.24.253.126|            1|
|  101.142.52.17|            1|
|   190.253.9.63|            1|
+---------------+-------------+



2. `Сгруппируйте данные по HTTP-методу и посчитайте количество запросов для каждого метода`

In [26]:
task2_df = (
    df
    .groupBy("method")
    .agg(count("*").alias("method_count"))
)

print("Request count by HTTP method:")
task2_df.show()

Request count by HTTP method:
+------+------------+
|method|method_count|
+------+------------+
|  POST|       25230|
|DELETE|       24851|
|   PUT|       24948|
|   GET|       24971|
+------+------------+



3. `Профильтруйте и посчитайте количество запросов с кодом ответа 404`

In [27]:
task3_df = (
    df
    .filter(col("response_code") == 404)
)
print(f" Numbers of 404 response codes: {task3_df.count()}")

 Numbers of 404 response codes: 24705


4. `Сгруппируйте данные по дате и просуммируйте размер ответов, сортируйте по дате.`

In [28]:
task4_df = (
    df
    .withColumn("timestamp", to_date(col("timestamp")))
    .groupBy("timestamp")
    .agg(sum(col("response_size")).alias("total_response_size"))
    .orderBy("timestamp")
)
task4_df.cache()
print("Total response size by day: ")
task4_df.show(10, 100, False)

Total response size by day: 


24/09/07 19:43:04 WARN CacheManager: Asked to cache already cached data.


+----------+-------------------+
| timestamp|total_response_size|
+----------+-------------------+
|2024-01-01|            1905525|
|2024-01-02|            1953726|
|2024-01-03|            2007048|
|2024-01-04|            1916647|
|2024-01-05|            2015058|
|2024-01-06|            2053226|
|2024-01-07|            2101514|
|2024-01-08|            2002937|
|2024-01-09|            2142065|
|2024-01-10|            2034381|
+----------+-------------------+
only showing top 10 rows

